In [1]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')


In [2]:
image = cv2.imread('colour.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
colour = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [3]:
gX = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
gY = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
gX = cv2.convertScaleAbs(gX)
gY = cv2.convertScaleAbs(gY)

In [4]:
cv2.imshow("Sobel/Scharr X", gX)
cv2.imshow("Sobel/Scharr Y", gY)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
blur = cv2.GaussianBlur(gray, (5,5), cv2.BORDER_DEFAULT)
cv2.imshow("Smoothing", blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
bX = cv2.Sobel(blur, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
bY = cv2.Sobel(blur, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
bX = cv2.convertScaleAbs(bX)
bY = cv2.convertScaleAbs(bY)

In [7]:
cv2.imshow("Sobel/Scharr X", bX)
cv2.imshow("Sobel/Scharr Y", bY)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
magnitude = np.sqrt((bX ** 2) + (bY ** 2))
orientation = np.arctan2(bY, bX) * (180 / np.pi) % 180

In [9]:
(fig, axs) = plt.subplots(nrows=1, ncols=3, figsize=(8, 4))
# plot each of the images
axs[0].imshow(blur, cmap="gray")
axs[1].imshow(magnitude, cmap="jet")
axs[2].imshow(orientation, cmap="jet")
# set the titles of each axes
axs[0].set_title("Grayscale")
axs[1].set_title("Gradient Magnitude")
axs[2].set_title("Gradient Orientation [0, 180]")
# loop over each of the axes and turn off the x and y ticks
for i in range(0, 3):
	axs[i].get_xaxis().set_ticks([])
	axs[i].get_yaxis().set_ticks([])
# show the plots
plt.tight_layout()
plt.show()

In [10]:
wide = cv2.Canny(blur, 10, 200)
mid = cv2.Canny(blur, 30, 150)
tight = cv2.Canny(blur, 240, 250)


In [11]:
cv2.imshow("Wide Edge Map", wide)
cv2.imshow("Mid Edge Map", mid)
cv2.imshow("Tight Edge Map", tight)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
contours, hierarchy = cv2.findContours(image=tight, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)

In [13]:
copy = blur.copy()
cv2.drawContours(blur, contours, -1, color=(0, 254, 0), thickness=3)
cv2.imshow('Contours', blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
print("Number of Contours found = " + str(len(contours))) 

Number of Contours found = 48


In [15]:
ret, thresh = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY)
cv2.imshow('Binary image', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [16]:
cont, hier = cv2.findContours(thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
image_copy = blur.copy()
cv2.drawContours(image=image_copy, contours=cont, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
cv2.imshow('None approximation', colour)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
print("Number of Contours found = " + str(len(cont))) 

Number of Contours found = 7


In [18]:
res = cv2.polylines(blur, cont[1], isClosed=True, color=(255, 0, 0), thickness=4)
print(res)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [19]:
cont_info = [(i, cv2.contourArea(i)) for i in cont]

In [20]:
mask = np.ones(colour.shape, dtype=np.uint8)
mask = cv2.fillConvexPoly(mask, cont[3], [0,0,0])
mult = np.multiply(colour, mask)


In [21]:
cv2.imshow('Mask', mult)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
print(cv2.contourArea(cont[1]))

890311.5


In [26]:
colour = cv2.fillConvexPoly(colour, cont[3], [255,255,0])
cv2.imshow('Mask', colour)
cv2.waitKey(0)
cv2.destroyAllWindows()